# Проект "Бурение скважин"

## Описание проекта

Добывающая компания «ГлавРосГосНефть» решает, где бурить новую скважину. Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Нужно построить модель для определения региона, где добыча принесёт наибольшую прибыль, а также проанализировать возможную прибыль и риски техникой Bootstrap.

## План выполнения проекта

1. [**Загрузите и подготовьте данные.**](#step1) Поясните порядок действий. Данные геологоразведки трёх регионов находятся в файлах: <i>/datasets/geo_data_0.csv, /datasets/geo_data_1.csv, /datasets/geo_data_2.csv</i>.
1. [**Обучите и проверьте модель для каждого региона:**](#step2)
    - [Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.](#step2_1)
    - [Обучите модель и сделайте предсказания на валидационной выборке.](#step2_2) Сохраните предсказания и правильные ответы на валидационной выборке.
    - [Напечатайте на экране средний запас предсказанного сырья и RMSE модели.](#step2_3)
    - [Проанализируйте результаты.](#step2_4)
1. [**Подготовьтесь к расчёту прибыли:**](#step3)
    - [Все ключевые значения для расчётов сохраните в отдельных переменных.](#step3_1)
    - [Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.](#step3_2)
    - [Напишите выводы по этапу подготовки расчёта прибыли.](#step3_3)
1. [**Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.**](#step4) Выберите скважины с максимальными значениями предсказаний, просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям и рассчитайте прибыль для полученного объёма сырья.
1. [**Посчитайте риски и прибыль для каждого региона:**](#step5)
    - [Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.](#step5_1)
    - [Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков (отрицательной прибыли).](#step5_2)
    - [Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.](#step5_3)

## Описание данных

- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три некоторых признака точек;
- `product` — объём запасов в скважине (тыс. баррелей).

## Условия задачи

- Для обучения модели подходит только линейная регрессия;
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки;
- Бюджет на разработку скважин в регионе — 10 млрд рублей;
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей;
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью;
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## <a name="step1"></a>Шаг 1. Загрузите и подготовьте данные

Для начала сделаем импорт необходимых библиотек.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy.random import RandomState
import scipy.stats as st

Откроем файлы с данными, назовём их по смыслу.

In [2]:
try:
    region1 = pd.read_csv('/Users/andreykol/PycharmProjects/Yandex.Praktikum-projects/project_ML_in_Business/geo_data_0.csv')
    #region1 = pd.read_csv('/datasets/geo_data_0.csv')
    region2 = pd.read_csv('/Users/andreykol/PycharmProjects/Yandex.Praktikum-projects/project_ML_in_Business/geo_data_1.csv')
    #region2 = pd.read_csv('/datasets/geo_data_1.csv')
    region3 = pd.read_csv('/Users/andreykol/PycharmProjects/Yandex.Praktikum-projects/project_ML_in_Business/geo_data_2.csv')
    #region3 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    print('Ошибка при чтении файла!')

Узнаем подробную информацию про наборы данных.

In [3]:
for i in range(1, 4):
    print(globals()['region'+str(i)].info())
    print(globals()['region'+str(i)].columns)
    display(globals()['region'+str(i)].head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.22117,105.280062
1,2acmU,1.334711,-0.340164,4.36508,73.037750


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697


Нигде нет пропусков, а структура таблиц ясна. Обучим модели для предсказания объёмов запасов.

## <a name="step2"></a>Шаг 2. Обучите и проверьте модель для каждого региона

### <a name="step2_1"></a>Разбейте данные на обучающую и валидационную выборки в соотношении 75:25

In [4]:
for i in range(1, 4):
    globals()['X'+str(i)] = globals()['region'+str(i)].drop(columns=['id', 'product'])
    globals()['y'+str(i)] = globals()['region'+str(i)]['product']
    (globals()['X'+str(i)+'_train'], globals()['X'+str(i)+'_valid'],
     globals()['y'+str(i)+'_train'], globals()['y'+str(i)+'_valid']) = (train_test_split(globals()['X'+str(i)], 
                                                                                       globals()['y'+str(i)],
                                                                            test_size=0.25, random_state=42))
# all names are given in the following form: 'Xi_train', 'yi_valid', where i is dataset's number

С помощью функции `globals()`, переводящей строку в название переменной, мы смогли быстро разделить каждый из трёх датасетов на четыре части (признаки `Xi_train` и таргет `yi_train` в обучающей выборке, они же (`Xi_valid`, `yi_valid`) в валидационной).

### <a name="step2_2"></a>Обучите модель и сделайте предсказания на валидационной выборке

По условию задачи, для каждого набора данных обучим линейную регрессию в качестве модели.

In [5]:
our_model = LinearRegression()
for i in range(1, 4):
    our_model.fit(globals()['X'+str(i)+'_train'], globals()['y'+str(i)+'_train'])
    globals()['y'+str(i)+'_pred'] = pd.Series(our_model.predict(globals()['X'+str(i)+'_valid']), 
                                              index=(globals()['X'+str(i)+'_valid']).index)

Итак, теперь в переменных `yi_pred` хранятся предсказания модели на валидационной выборке.

### <a name="step2_3"></a>Напечатайте на экране средний запас предсказанного сырья и RMSE модели

Отыщем средние значения запасов сырья, а также подсчитаем метрику RMSE (корень из среднеквадратического отклонения) в каждом регионе.

In [6]:
for i in range(1, 4):
    globals()['mean'+str(i)] = globals()['y'+str(i)+'_pred'].mean()
    globals()['rmse'+str(i)] = sqrt(mean_squared_error(globals()['y'+str(i)+'_valid'], globals()['y'+str(i)+'_pred']))
    print('Регион {}: средний запас равен {}, RMSE модели равен {}'
          .format(i, globals()['mean'+str(i)], globals()['rmse'+str(i)]))

Регион 1: средний запас равен 92.39879990657768, RMSE модели равен 37.75660035026169
Регион 2: средний запас равен 68.71287803913764, RMSE модели равен 0.8902801001028828
Регион 3: средний запас равен 94.77102387765939, RMSE модели равен 40.145872311342174


Получили, что средний запас в регионах оценивается в 92, 68 и 94 тысячи баррелей соответственно. Ошибки моделей в регионах 1 и 3 примерно совпадают, а модель для региона 2 предсказала данные очень точно.

### <a name="step2_4"></a>Проанализируйте результаты

Итак, на данном этапе исследования мы знаем оценку средних запасов сырья в каждом регионе, а также степень ошибки этих предсказаний. Становится ясно, что мы можем быть достаточно уверены в оценке объёмов сырья во втором регионе. Однако, среднее значение там минимально. Почти в полтора раза выше этот показатель в регионах 1 и 3. Но теперь мы не можем быть очень уверены в том, что оценка точна, ведь метрика RMSE далека от нуля. Результаты регионов 1 и 3 очень похожи. Да и отбирать мы будем только самые перспективные скважины, поэтому среднее по всем скважинам нам пока мало что даёт.

## <a name="step3"></a>Подготовка к расчёту прибыли

### <a name="step3_1"></a>Все ключевые значения для расчётов сохраните в отдельных переменных

Некоторые важные значения были сохранены отдельно в предыдущих пунктах (`meani` для средних значений и `rmsei` для ошибок моделей).

In [7]:
to_drill = 200 # количество скважин для бурения
budget = 10e9 # общий бюджет
gain = 450e3 # доход с каждой единицы продукта

### <a name="step3_2"></a>Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе

In [8]:
suff_amount = (budget / to_drill) / gain
print(suff_amount)

111.11111111111111


Мы получили, что для безубыточной разработки новой скважины необходимо, чтобы объём сырья составлял около 111 тысяч баррелей. Очевидно, что в каждом из регионов средние значения объёмов сырья меньше целевого значения (они были равны 92, 68 и 94 тысячам соответственно).

### <a name="step3_3"></a>Напишите выводы по этапу подготовки расчёта прибыли

На основании сделанных вычислений можно сказать, что исходя лишь из средних значений добычи, нам не удастся выйти на безубыточную разработку новой скважины. Однако важно понимать, что это всего лишь средние значения среди всех новых скважин, надо учесть и разброс значений. Тем более, позже мы отберём самые "перспективные" точки, по ним среднее значения будет выше.

## <a name="step4"></a>Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [9]:
def count_profit(wells_boot):
    best_200 = wells_boot.sort_values(ascending=False)[:200]
    sum_amount = best_200.sum()
    profit = sum_amount * gain - budget
    return profit

С помощью сортировки, мы сначала отобрали топ-200 скважин по объёмам сырья, затем просуммировали объёмы и нашли прибыль, умножив объём добычи на доход с каждой единицы продукта.

## <a name="step5"></a>Посчитайте риски и прибыль для каждого региона

### <a name="step5_1"></a>Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли

In [10]:
state = RandomState(42)
boots = 1000
for i in range(1, 4):
    globals()['profits'+str(i)] = []
    for j in range(boots):
        one_sample = globals()['y'+str(i)+'_pred'].sample(n=500, replace=False, random_state=state)
        globals()['profits'+str(i)].append(count_profit(one_sample))
    globals()['profits'+str(i)] = pd.Series(globals()['profits'+str(i)])

Итак, в переменных `profitsi` теперь у нас содержатся значения прибыли на подвыборках (три группы по 1000 выборок).

### <a name="step5_2"></a>Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков

In [11]:
conf_level = 0.95
stand_dist = st.t.ppf((1 + conf_level) / 2., boots-1)
# это число, умнож. на станд. ошибку среднего, даст расстояние от конца дов. инт. до среднего 
# (точка, соотв. 97.5%), почти как у нормального распр., так как имеем 999 степ. свободы
for i in range(1, 4):
    globals()['mean_profit'+str(i)] = globals()['profits'+str(i)].mean()
    globals()['sem_profit'+str(i)] = globals()['profits'+str(i)].sem()
    globals()['conf_int_profit'+str(i)] = [globals()['mean_profit'+str(i)] - stand_dist * globals()['sem_profit'+str(i)], 
                                           globals()['mean_profit'+str(i)] + stand_dist * globals()['sem_profit'+str(i)]]
    globals()['loss_risk'+str(i)] = st.percentileofscore(globals()['profits'+str(i)], 0) / 100
    # доля средних, меньших 10 млрд

In [14]:
for i in range(1, 4):
    print('Регион {}:'.format(i))
    print('Средняя прибыль (млн руб.): {}'.format(np.asarray(globals()['mean_profit'+str(i)]) / 1e6))
    print('95%-й дов. инт. (млн руб.): {}'.format(np.asarray(globals()['conf_int_profit'+str(i)]) / 1e6))
    print('Риск убытков: {:.1%}'.format(globals()['loss_risk'+str(i)]))

Регион 1:
Средняя прибыль (млн руб.): 330.4344959945319
95%-й дов. инт. (млн руб.): [323.46296912 337.40602287]
Риск убытков: 0.4%
Регион 2:
Средняя прибыль (млн руб.): 433.5392331313466
95%-й дов. инт. (млн руб.): [420.67870282 446.39976344]
Риск убытков: 1.8%
Регион 3:
Средняя прибыль (млн руб.): 267.77206957956736
95%-й дов. инт. (млн руб.): [261.51240321 274.03173595]
Риск убытков: 0.3%


### <a name="step5_3"></a>Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор

Итак, исходя из финальных результатов, мы обнаружили, что все три региона безопасности в плане риска убытков (везде менее 2.5%). Следующим критерием выбора региона является средняя прибыль. Она выше всего в регионе 2 (433 против 330 и 267). Поэтому, анализ показывает, что аналитически вполне обоснованным решением является разработка скважин в регионе 2.